As a brief background:

I want to gather scores for a subset of TikTok data to compare the quality of our models - Appen managed services - Our moderator team

On meta, this is an easy task, as there is infrastructure built in our resolvers to do this.

This does not exist for TikTok.

I created a protocol that could possibly facilitate the scoring of arbitrary TikTok content.

The problem?

*There are no nodes that do frame extraction/thumbnails for TikTok videos*
This is a notebook to bridge that gap.

Input: TikTok content with Metadata/videos downloaded
Output: TikTok content with frames extracted and "content_type" set to "VIDEO"


In [1]:
import itertools
import time
import json
from typing import Iterable

from qa_frame.consts.api import frs, apr
from qa_frame.consts.providers.s3 import SamplePostsCSV
from qa_frame.interfaces.api.clients.frs import FRSClient
from qa_frame.interfaces.api.clients.seldon import SeldonClient
from qa_frame.interfaces.api.clients.apr import APRClient
from qa_frame.interfaces.db.snowflake import SnowflakeDriver
from qa_frame.interfaces.external.s3 import S3Client
from qa_frame.models.avro.schemas.video import Content, Video, ContentType, Derived
from qa_frame.models.environment import EnvConfig
from qa_frame.models.util import JSONType
from requests import RequestException

from base import BaseScript

CONFIG = EnvConfig.get()

In [ ]:
"""
Gathers TikTok video IDs that need to be scored
"""
s3 = S3Client(CONFIG.external.s3_client)
tiktok_video_ids = s3.read_csv(
    url=str(SamplePostsCSV.AMS_HPQ_TT_DATA.value),
    filter_=lambda csv_: (
        Video(**{"content_id": row["content_id"][1:-1]})
        for row in csv_
    )
)  # Formatted as video.Video objects


In [ ]:
"""
Gathers metadata for our video IDs

Must port-forward the TikTok metadata service
"""

class GatherTiktokMetadataForVideos(BaseScript):

    name = "gather_tiktok_metadata"

    def __init__(self):
        super().__init__()
        self.client = SeldonClient[Video, Video](base_url="http://localhost:8080/api/v1.0")

    def run(self):
        batch_size = 50
        scored_content_count = 0

        while batch := list(itertools.islice(tiktok_video_ids, batch_size)):
            predictions, retry = [], True
            while retry:
                try:
                    predictions = [v for v in self.client.predictions(batch) if v.metadata_fetch_status == 200]
                    scored_content_count += len(predictions)
                    retry = False
                except RequestException:
                    continue

            self._write_json_files("tiktok_videos_with_metadata", *predictions)
            self.logger.info("Predictions received", total_count=scored_content_count)

GatherTiktokMetadataForVideos().run()

In [ ]:
"""
DownloadTikTokVideos
"""

class DownloadTikTokVideos(BaseScript):
    name = "download_tiktok_videos"

    def __init__(self):
        super().__init__()
        self.client = SeldonClient[Video, Video](base_url="http://localhost:8080/api/v1.0")

    def run(self):
        batch_size = 50
        scored_content_count = 0

        videos_with_metadata = self._read_json_files("tiktok_videos_with_metadata_1675844107", Video)
        while batch := list(itertools.islice(videos_with_metadata, batch_size)):
            predictions, retry = [], True
            while retry:
                try:
                    predictions = self.client.predictions(batch)
                    scored_content_count += len(predictions)
                    retry = False
                except RequestException:
                    continue

            self._write_json_files("tiktok_videos_downloaded", *predictions)
            self.logger.info("Predictions received", total_count=scored_content_count)

DownloadTikTokVideos().run()

In [ ]:
"""
Merge together metadata and download url
"""

class MergeTiktokMetadataAndDownloadedVideo(BaseScript):  # Also will add a content_type
    name = "merge_tiktok_videos"

    def run(self):
        for video_metadata in self._read_json_files("tiktok_videos_with_metadata_1675844107", Content):
            corresponding_downloaded_video = next(
                v
                for v in self._read_json_files("tiktok_videos_downloaded_1675910561", Video)
                if v.content_id == video_metadata.content_id
            )
            merged_video = Content(**video_metadata.as_dict() | {"content_type": ContentType.VIDEO, "video_url": corresponding_downloaded_video.video_url})
            self._write_json_files("videos", merged_video)

MergeTiktokMetadataAndDownloadedVideo().run()

In [ ]:
"""
Extract frames from merged videos
"""
from threading import Thread
from concurrent.futures import ThreadPoolExecutor, wait
from queue import Queue
from requests import ConnectionError, Session
from requests.adapters import HTTPAdapter
from time import sleep


max_video_seconds = 60
num_threads = 32


class ExtractTiktokVideoFrames(BaseScript):
    name = "extract_tiktok_video_frames"

    def __init__(self):
        super().__init__()
        self.session = Session()
        self.session.mount("http://", HTTPAdapter(pool_connections=num_threads, pool_maxsize=num_threads))
        self.frs = FRSClient("http://localhost:8080", session=self.session)
        self.contents = self._read_json_files("videos_1675913568", Content)
        self.contents_to_write = Queue[Content]()

    def write_to_file(self):
        while True:
            if not self.contents_to_write.empty():
                self._write_json_files("videos_with_frames", self.contents_to_write.get())

    def get_frames(self, content: Content):
        retry = True
        retry_count = 0

        while retry:
            try:
                response = self.frs.store_video_frames(
                    video_uri=content.video_url,
                    platform=frs.Platform.TIKTOK,
                    content_id=content.content_id,
                    max_seconds=max_video_seconds
                )
                retry = False
                self.contents_to_write.put(Content(**content.as_dict() | {"derived": Derived(**response["derived"]), "thumbnail": None}))
            except ConnectionError:
                print("connection error, retrying in 10")
                sleep(10)
            except RequestException:
                if retry_count == 2:
                    return
                retry_count += 1
                print("request exception, waiting 10 and retrying")
                sleep(10)

    def run(self):
        content_writer = Thread(target=self.write_to_file)
        content_writer.daemon = True
        content_writer.start()

        with ThreadPoolExecutor(max_workers=num_threads) as thread_pool:
            futures = thread_pool.map(self.get_frames, self._read_json_files("videos_1675913568", Content))
            wait(futures)
        content_writer.join()

ExtractTiktokVideoFrames().run()

In [ ]:
class RunTikTokVideosThroughModel(BaseScript):
    name = "scored_tiktok_videos"
    model_name = "seldon-meta-combiner-predictor"

    def __init__(self):
        super().__init__()
        self.client = SeldonClient[Content, Content](
            base_url="http://localhost:8080/api/v1.0"
        )

    def run(self):
        batch_size = 50
        scored_content_count = 0

        videos = self._read_json_files("YTCR-Meta-Combiner-Combined_1677918254", Content)
        while batch := list(itertools.islice(videos, batch_size)):
            predictions, retry = [], True
            while retry:
                try:
                    predictions = self.client.predictions(batch)
                    scored_content_count += len(predictions)
                    self.logger.info("Predictions received", total_count=scored_content_count)
                    retry = False
                except RequestException:
                    continue

            self._write_json_files(self.model_name, *predictions)

RunTikTokVideosThroughModel().run()

In [3]:
# noinspection SqlResolve
class GetExistingRunContent(SnowflakeDriver):
    def read_videos(self, existing_run_id: int) -> Iterable[dict[str, str]]:
        yield from (Video(**json.loads(row["OUTPUT_DATA"])) for row in self._read("""
            SELECT OUTPUT_DATA
            FROM STAGE.STAGING.QA_FRAME_INFERENCE_COMPARE
            WHERE RUN_ID = %s
        """, existing_run_id))


class RunExistingRunContentThroughModel(BaseScript):
    name = "RunExistingRunContentThroughModel"
    model_name = "tt_cr"
    existing_run_id = "1677867854"

    def __init__(self):
        super().__init__()
        self.client = SeldonClient[Video, JSONType](
            base_url="https://models.stage.zefr.com/seldon/tiktok-models/seldon-tt-combiner/api/v1.0/",
        )
        self.videos = GetExistingRunContent(self.snowflake_connection).read_videos(self.existing_run_id)

    def run(self):
        batch_size = 50
        scored_content_count = 0

        while batch := list(itertools.islice(self.videos, batch_size)):
            predictions, retry = [], True

            # clean up
            for i, video in enumerate(batch):
                vid = video.as_dict()
                vid["derived"]["taxonomy_scores"] = None
                batch[i] = Video(**vid)

            while retry:
                try:
                    predictions = [
                        Video(
                            content_id=v["content_id"],
                            derived=Derived(taxonomy_scores={k: v | {"score": float(v["score"])} for k, v in v["taxonomy_scores"].items()})
                        )
                        for v in self.client.predictions(batch)
                    ]
                    scored_content_count += len(predictions)
                    self.logger.info("Predictions received", total_count=scored_content_count)
                    retry = False
                except (RequestException, ConnectionError):
                    time.sleep(5)
                    continue
            self._write_json_files(self.model_name, *predictions)

RunExistingRunContentThroughModel().run()


{
  "asctime": "2023-03-10 07:16:18",
  "levelname": "INFO",
  "lineno": 54,
  "msg": "Initialized output directory",
  "output_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/output/RunExistingRunContentThroughModel"
}

{
  "asctime": "2023-03-10 07:16:19",
  "levelname": "INFO",
  "lineno": 79,
  "msg": "Connected to snowflake",
  "connection": {
    "host": "127.0.0.1",
    "port": "10017",
    "protocol": "http",
    "user": "ryandem",
    "account": "any",
    "database": "PROD",
    "warehouse": "ENGINEER_WH",
    "schema_": null
  }
}

{
  "asctime": "2023-03-10 07:16:54",
  "levelname": "INFO",
  "lineno": 46,
  "msg": "Predictions received",
  "total_count": 50
}

{
  "asctime": "2023-03-10 07:16:54",
  "levelname": "INFO",
  "lineno": 114,
  "msg": "Writing file",
  "file_name": "tt_cr_0.json"
}

{
  "asctime": "2023-03-10 07:17:13",
  "levelname": "INFO",
  "lineno": 46,
  "msg": "Predictions received",
  "total_count": 100
}

{
  "asctime": "2023-

KeyboardInterrupt: 

In [8]:
# noinspection SqlResolve
class GetExistingRunContentIDs(SnowflakeDriver):
    def read_content_ids(self, existing_run_id: int) -> Iterable[dict[str, str]]:
        yield from ({"content_id": row["CONTENT_ID"][1:-1]} for row in self._read("""
            SELECT OUTPUT_DATA:content_id as CONTENT_ID
            FROM STAGE.STAGING.QA_FRAME_INFERENCE_COMPARE
            WHERE RUN_ID = %s
        """, existing_run_id))


class RunExistingRunContentThroughResolver(BaseScript):
    name = "RunExistingRunContentThroughResolver"
    resolver_name = "MetaDiscoveryTier2"
    existing_run_id = "1677538266"

    def __init__(self):
        super().__init__()
        self.resolver = APRClient[Content, Content](
            base_url="https://models.zefr.com/resolver/meta-discovery",
            protocol=apr.APRProtocol.MetaDiscoveryTier2
        )
        self.content_ids = GetExistingRunContentIDs(self.snowflake_connection).read_content_ids(self.existing_run_id)

    def run(self):
        batch_size = 50
        scored_content_count = 0

        input_type, _ = self.resolver.io_types

        while batch := [input_type(**row) for row in itertools.islice(self.content_ids, batch_size)]:
            scores = self.resolver.resolve_best_effort(batch)
            scored_content_count += len(scores)

            self._write_json_files(self.resolver_name, *scores)
            self.logger.info("Scores received", total_count=scored_content_count)

RunExistingRunContentThroughResolver().run()

{
  "asctime": "2023-03-10 12:37:16",
  "levelname": "INFO",
  "lineno": 54,
  "msg": "Initialized output directory",
  "output_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/output/RunExistingRunContentThroughResolver"
}

{
  "asctime": "2023-03-10 12:37:17",
  "levelname": "INFO",
  "lineno": 79,
  "msg": "Connected to snowflake",
  "connection": {
    "host": "127.0.0.1",
    "port": "10017",
    "protocol": "http",
    "user": "ryandem",
    "account": "any",
    "database": "PROD",
    "warehouse": "ENGINEER_WH",
    "schema_": null
  }
}

{
  "asctime": "2023-03-10 12:37:56",
  "levelname": "INFO",
  "lineno": 114,
  "msg": "Writing file",
  "file_name": "MetaDiscoveryTier2_0.json"
}

{
  "asctime": "2023-03-10 12:37:56",
  "levelname": "INFO",
  "lineno": 35,
  "msg": "Scores received",
  "total_count": 50
}

{
  "asctime": "2023-03-10 12:38:33",
  "levelname": "INFO",
  "lineno": 35,
  "msg": "Scores received",
  "total_count": 100
}

{
  "asctime": 

KeyboardInterrupt: 

In [ ]:
class CombineScores(BaseScript):
    name = "CombineScores"
    content_folder_1_name = "YTCR-Meta-Combiner-Combined_1677918254"
    content_folder_2_name = "seldon-meta-combiner-predictor_1677918861"

    def run(self):
        contents_1: dict[str, Content] = {c.content_id: c for c in self._read_json_files(self.content_folder_1_name, Content)}
        contents_2: dict[str, Content] = {c.content_id: c for c in self._read_json_files(self.content_folder_2_name, Content)}

        for content_id, content in contents_1.items():
            corr_content = contents_2[content_id]
            combined = Content(content_id=content_id, derived=Derived(taxonomy_scores=content.derived.taxonomy_scores | corr_content.derived.taxonomy_scores), title=content.title)
            # combined = Content(content_id=content_id, title=content.title, derived=corr_content.derived)

            self._write_json_files("YT-CR_Meta-Combiner_Combined", combined)

CombineScores().run()

In [ ]:
class GetExistingRunContent(SnowflakeDriver):
    def read_content_ids(self, existing_run_id: int) -> Iterable[dict[str, str]]:
        yield from (Video(**json.loads(row["OUTPUT_DATA"])) for row in self._read("""
            SELECT OUTPUT_DATA
            FROM STAGE.STAGING.QA_FRAME_INFERENCE_COMPARE
            WHERE RUN_ID = %s
        """, existing_run_id))

class GetExistingRunJsons(BaseScript):
    name = "GetExistingRunJsons"
    existing_run_id = "1677538266"

    def __init__(self):
        super().__init__()
        self.videos = GetExistingRunContent(self.snowflake_connection).read_content_ids(self.existing_run_id)

    def run(self):
        for vid in self.videos:
            self._write_json_files("meta_hpq_with_titles", vid)

GetExistingRunJsons().run()